In [1]:
from settings import *
from init_state import read_initial_state
import policies
import policies.gleditsch_hagen
import policies.inngjerdingen_moeller
import policies.hlm
from helpers import timeInMinutes

# from output.plots import cityTrafficStats
import folium

state = read_initial_state(sb_jsonFilename = "instances/ebike_with_depot/TD_W34", ff_jsonFilename="instances/Ryde/TR_random_100")

state.set_seed(1)

policy = policies.hlm.BS_PILOT()

state.set_sb_vehicles([policy]) # this creates one vehicle for each policy in the list



In [2]:

# Create a map centered around a default location
mymap = folium.Map(location=[63.446827, 10.421906], zoom_start=13)

# Plot the hexagons on the map
for hex in state.get_areas():
    # Create a polygon and add to the map
    folium.Polygon(locations=hex.border_vertices, fill=True, color="#ff7800", weight=2).add_to(mymap)
    for bike in hex.get_bikes():
        folium.CircleMarker(location=[bike.get_lat(), bike.get_lon()], radius=2, color='green', fill=True, fill_color='red').add_to(mymap)


# Plot trip coordinates as markers on the map
for station in state.get_stations():
    #folium.Marker(location=[lat, lng], popup=trip['trip_id'], icon=folium.Icon(color='blue')).add_to(mymap)
    folium.CircleMarker(location=[station.get_lat(), station.get_lon()], radius=2, color='red', fill=True, fill_color='red').add_to(mymap)

mymap

In [3]:
import sim
import demand
import target_state

tstate = target_state.USTargetState()
# tstate = target_state.HalfCapacityTargetState()

###############################################################################
# Set up demand

dmand = demand.demand.Demand()

###############################################################################
# Set up simulator


In [4]:
import sim
import demand
import target_state

simulator = sim.Simulator(
    initial_state = state,
    target_state = tstate,
    demand = dmand,
    start_time = timeInMinutes(hours=7),
    duration = timeInMinutes(hours=120),
    verbose = True,
)

simulator.run()

Running Sim |████████████████████████████████| 100% - ETA 0s


In [7]:

# Create a map centered around a default location
mymap = folium.Map(location=[63.446827, 10.421906], zoom_start=13)

# Plot the hexagons on the map
for hex in simulator.state.get_areas():
    # Create a polygon and add to the map
    if hex.number_of_bikes() > 0:
        folium.Polygon(locations=hex.border_vertices, fill=True, color="black", weight=2, popup=str(hex.number_of_bikes())).add_to(mymap)
    else:
        folium.Polygon(locations=hex.border_vertices, fill=True, color="#ff7800", weight=2, popup=str(hex.number_of_bikes())).add_to(mymap)
    for bike in hex.get_bikes():
        folium.CircleMarker(location=[bike.get_lat(), bike.get_lon()], radius=2, color='green', fill=True, fill_color='red').add_to(mymap)


# Plot trip coordinates as markers on the map
for station in simulator.state.get_stations():
    #folium.Marker(location=[lat, lng], popup=trip['trip_id'], icon=folium.Icon(color='blue')).add_to(mymap)
    folium.CircleMarker(location=[station.get_lat(), station.get_lon()], radius=2, color='red', fill=True, fill_color='red', popup=str(len(station.get_bikes()))).add_to(mymap)

mymap